In [23]:
import pandas as pd

from statistics import mean
import statistics
import numpy as np
import os
import subprocess
import re

In [24]:
results = []

import json
with open("ast_results.json", "r") as fd:
    results = json.load(fd)
    
with open("r2-retdec_ast_results.json", "r") as fd:
    data = json.load(fd)
    
for r in results:
    has_set = False
    for d in data:
        if d["opt"] == r["opt"] and d["filename"] == r['filename']:
            for k in r["results"]:
                if k in d["results"]:
                    r['results'][k].update(d['results'][k])
                else:
                    r['results'][k]['r2-ghidra'] = {'nqc': -1, 'ted': -1}
                    r['results'][k]['retdec'] = {'nqc': -1, 'ted': -1}
            has_set = True
    if not has_set:
        for k in r["results"]:
            print(k)
            r['results'][k]['r2-ghidra'] = {'nqc': -1, 'ted': -1}
            r['results'][k]['retdec'] = {'nqc': -1, 'ted': -1}

In [25]:
results[0]

{'opt': 0,
 'filename': 'PL_2mm',
 'results': {'kernel_2mm': {'w2c2': {'nqc': 0.015300023906287354,
    'ted': 0.031051517290049402},
   'wasm2c': {'nqc': 0.01562881562881563, 'ted': 0.017295219793418193},
   'r2-ghidra': {'nqc': 0.1933534743202417, 'ted': 0.22556390977443608},
   'retdec': {'nqc': 0.029465930018416207, 'ted': 0.042857142857142816}}}}

In [42]:
func_results = {}


for data in results:
    opt = data["opt"]
    filename = data["filename"]
    for func, result in data["results"].items():
        func_name = f'{filename}:{func}:O{opt}'
        if func_name not in func_results:
            func_results[func_name] = {}
        for tool in ["w2c2", "wasm2c", "r2-ghidra", 'retdec']:
            try:
                func_results[func_name][tool] = { "opt": int(opt),
                                 "nqc": result[tool]["nqc"], 
                                 "ted": result[tool]["ted"]}
            except:
                print(opt, tool, func_name, result)
                print()
            #if result[tool]["ted"]< 0 and opt != 2:
            #    print(opt, tool, func_name)

func_names = []
rows = []
for func_name, columns in func_results.items():
    func_names.append(func_name)
    rows.append(columns)
        
d = pd.json_normalize(rows)
df = pd.DataFrame(d)
df.columns = df.columns.str.split('.', expand=True)
# print(df.loc[0])
df["func"] = func_names
df = df.set_index("func")

pd.set_option('display.max_rows', None)
display(df['r2-ghidra'])
pd.reset_option('display.max_rows')

,opt,nqc,ted
func,,,
PL_2mm:kernel_2mm:O0,0,0.193353,0.225564
PL_3mm:kernel_3mm:O0,0,0.201735,0.228571
adi:kernel_adi:O0,0,0.216651,0.242146
adpcm:abs:O0,0,0.875000,0.711111
adpcm:encode:O0,0,0.459883,0.400000
adpcm:filtez:O0,0,0.506024,0.613139
adpcm:filtep:O0,0,0.581395,0.260870
adpcm:quantl:O0,0,0.652632,0.494118
adpcm:logscl:O0,0,0.830189,0.613861


In [43]:
print("TOs / Nil")
for opt in [0,1,2]:
    for tool in ["w2c2", "wasm2c", "r2-ghidra", 'retdec']:
        cnt = df[tool][(df[tool]["opt"] == opt) & (df[tool]["ted"] < 0)]["ted"].count()
        print(opt, tool, cnt)

print()
        
print("Presented data")
for opt in [0,1,2]:
    for tool in ["w2c2", "wasm2c", "r2-ghidra", 'retdec']:
        cnt = df[tool][(df[tool]["opt"] == opt) & (df[tool]["ted"] > 0)]["ted"].count()
        print(opt, tool, cnt)

TOs / Nil
0 w2c2 17
0 wasm2c 29
0 r2-ghidra 94
0 retdec 94
1 w2c2 14
1 wasm2c 16
1 r2-ghidra 108
1 retdec 108
2 w2c2 15
2 wasm2c 17
2 r2-ghidra 126
2 retdec 126

Presented data
0 w2c2 130
0 wasm2c 118
0 r2-ghidra 53
0 retdec 53
1 w2c2 133
1 wasm2c 131
1 r2-ghidra 39
1 retdec 39
2 w2c2 132
2 wasm2c 130
2 r2-ghidra 21
2 retdec 21


In [8]:
from tabulate import tabulate

table = [
    ["Tool", "Opt", "Total func", "NQC (avg)", "NQC (stdev)", "TED (avg)", "TED (stdev)", "TED (TO)"]
]
for tool in ["w2c2", "wasm2c", "r2-ghidra", 'retdec']:
    for opt in [0,1,2]:
        opt_tool_df = df[tool][(df[tool]["opt"] == opt)]
        cnt = opt_tool_df["opt"].count()
        avg_nqc = opt_tool_df[(opt_tool_df["nqc"] > 0)]["nqc"].mean()
        std_nqc = opt_tool_df[(opt_tool_df["nqc"] > 0)]["nqc"].std()
        avg_ted = opt_tool_df[(opt_tool_df["ted"] > 0)]["ted"].mean()
        std_ted = opt_tool_df[(opt_tool_df["ted"] > 0)]["ted"].std()
        cnt_ted = opt_tool_df[(opt_tool_df["ted"] < 0)]["ted"].count()
        table.append(
            [tool, opt, cnt, avg_nqc,  std_nqc, avg_ted, std_ted, cnt_ted]
        )
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))        

╒═══════════╤═══════╤══════════════╤═════════════╤═══════════════╤═════════════╤═══════════════╤════════════╕
│ Tool      │   Opt │   Total func │   NQC (avg) │   NQC (stdev) │   TED (avg) │   TED (stdev) │   TED (TO) │
╞═══════════╪═══════╪══════════════╪═════════════╪═══════════════╪═════════════╪═══════════════╪════════════╡
│ w2c2      │     0 │          147 │   0.16846   │     0.205529  │   0.182024  │     0.172026  │         17 │
├───────────┼───────┼──────────────┼─────────────┼───────────────┼─────────────┼───────────────┼────────────┤
│ w2c2      │     1 │          147 │   0.229202  │     0.194485  │   0.231683  │     0.157311  │         14 │
├───────────┼───────┼──────────────┼─────────────┼───────────────┼─────────────┼───────────────┼────────────┤
│ w2c2      │     2 │          147 │   0.22258   │     0.19972   │   0.222599  │     0.167707  │         15 │
├───────────┼───────┼──────────────┼─────────────┼───────────────┼─────────────┼───────────────┼────────────┤
│ wasm2c  

In [13]:
# Hmm not sure what this is
# First seperate OPT level and make sure all have the same entries

opt_results = {}
for data in results:
    opt = data["opt"]
    filename = data["filename"]
    for func, result in data["results"].items():
        for tool in ["w2c2", "wasm2c", 'r2-ghidra', 'retdec']:
            if (opt, tool) not in opt_results:
                opt_results[(opt, tool)] = []
            opt_results[(opt, tool)].append(f"{filename}:{func}")
    